In [1]:
# Standard modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.time import Time
import progressbar

# Custom modules
from jpm_time_conversions import *
from jpm_logger import JpmLogger
from closest import closest
from jpm_number_printing import latex_float
from get_goes_flare_events import get_goes_flare_events
from determine_preflare_irradiance import determine_preflare_irradiance

__author__ = 'James Paul Mason'
__contact__ = 'jmason86@gmail.com'


def generate_jedi_catalog(threshold_time_prior_flare_minutes=240.0, 
                          dimming_window_relative_to_flare_minutes_left=0.0, 
                          dimming_window_relative_to_flare_minutes_right=240.0,
                          output_path='./', verbose=False):
    """Wrapper code for creating James's Extreme Ultraviolet Variability Experiment (EVE) Dimming Index (JEDI) catalog.

    Inputs:
        None.
        
    Optional Inputs:
        threshold_time_prior_flare_minutes [float]:             How long before a particular event does the last one need to have
                                                                occurred to be considered independent. If the previous one was too
                                                                recent, will use that event's pre-flare irradiance.
                                                                Default is 240 (4 hours).
        dimming_window_relative_to_flare_minutes_left [float]:  Defines the left side of the time window to search for dimming
                                                                relative to the GOES/XRS flare peak. Negative numbers mean 
                                                                minutes prior to the flare peak. Default is 0.0.
        dimming_window_relative_to_flare_minutes_right [float]: Defines the right side of the time window to search for dimming
                                                                relative to the GOES/XRS flare peak. If another flare
                                                                occurs before this, that time will define the end of the
                                                                window instead. Default is 240 (4 hours).
        output_path [str]:                                      Set to a path for saving the JEDI catalog table and processing 
                                                                summary plots. Default is './', the current directory.
        verbose [bool]:                                         Set to log the processing messages to disk and console. Default is False.

    Outputs:
        No direct return, but writes a (csv? sql table? hdf5?) to disk with the dimming paramerization results.
        Subroutines also optionally save processing plots to disk in output_path.
                                                 
    Optional Outputs:
        None

    Example:
        generate_jedi_catalog(output_path='/Users/jmason86/Dropbox/Research/Postdoc_NASA/Analysis/Coronal Dimming Analysis/JEDI Catalog/',
                              verbose=True)
    """
    return 1

In [2]:
# prototype only: set up example inputs
%matplotlib inline
plt.style.use('jpm-transparent-light')

threshold_time_prior_flare_minutes = 240.0
dimming_window_relative_to_flare_minutes_left = 0.0 
dimming_window_relative_to_flare_minutes_right = 240.0
output_path='/Users/jmason86/Dropbox/Research/Postdoc_NASA/Analysis/Coronal Dimming Analysis/JEDI Catalog/'
verbose = True

In [3]:
# Prepare the logger for verbose
if verbose:
    logger = JpmLogger(filename='generate_jedi_catalog', path=output_path)
    logger.info("Starting JEDI processing pipeline.")

2018-01-25 16:30:19,264 [INFO ] [<module>] Starting JEDI processing pipeline.


In [4]:
# Get EVE level 2 extracted emission lines data
# TODO: Replace this shortcut method with the method I'm building into sunpy
from scipy.io.idl import readsav
eve_readsav = readsav('/Users/jmason86/Dropbox/Research/Data/EVE/eve_lines_2010121-2014146 MEGS-A Mission Bare Bones.sav')

In [5]:
# prototype only: create metadata dictionary
# TODO: Replace this shortcut method with the method I'm building into sunpy
from sunpy.util.metadata import MetaDict
metadata = MetaDict()
metadata['ion'] = eve_readsav['name']
metadata['temperature_ion_peak_formation'] = np.power(10.0, eve_readsav['logt']) * u.Kelvin
metadata['extracted_wavelength_center'] = eve_readsav['wavelength'] * u.nm
metadata['extracted_wavelength_min'] = metadata['extracted_wavelength_center']
metadata['extracted_wavelength_max'] = metadata['extracted_wavelength_center']
metadata['emission_line_blends'] = ['none', 'yay', 'poop', 'Fe vi']  # etc
metadata['exposure_time'] = 60.0 * u.second  # These example EVE data are already binned down to 1 minute
metadata['precision'] = ['Not implemented in prototype']
metadata['accuracy'] = ['Not implemented in prototype']
metadata['flags'] = ['Not implemented in prototype']
metadata['flags_description'] = "1 = MEGS-A data is missing, 2 = MEGS-B data is missing, 4 = ESP data is missing, 8 = MEGS-P data is missing, 16 = Possible clock adjust in MEGS-A, 32 = Possible clock adjust in MEGS-B, 64 = Possible clock adjust in ESP, 128 = Possible clock adjust in MEGS-P"
metadata['flags_spacecraft'] = ['Not implemented in prototype']
metadata['flags_spacecraft_description'] = "0 = No obstruction, 1 = Warmup from Earth eclipse, 2 = Obstruction atmosphere penumbra, 3 = Obstruction atmosphere umbra, 4 = Obstruction penumbra of Mercury, 5 = Obstruction penumbra of Mercury, 6 = Obstruction penumbra of Venus, 7 = Obstruction umbra of Venus, 8 = Obstruction penumbra of Moon, 9 = Obstruction umbra of Moon, 10 = Obstruction penumbra of solid Earth, 11 = Obstruction umbra of solid Earth, 16 = Observatory is off-pointed by more than 1 arcmin"
metadata['data_version'] = ['Not implemented in prototype']
metadata['data_reprocessed_revision'] = ['Not implemented in prototype']
metadata['filename'] = ['Not implemented in prototype']

In [6]:
# prototype only: load up the actual irradiance data into a pandas DataFrame
# TODO: Replace this shortcut method with the method I'm building into sunpy
irradiance = eve_readsav['irradiance'].byteswap().newbyteorder()  # pandas doesn't like big endian
irradiance[irradiance == -1] = np.nan
wavelengths = eve_readsav['wavelength']
wavelengths_str = []
[wavelengths_str.append('{0:1.1f}'.format(wavelength)) for wavelength in wavelengths]
eve_lines = pd.DataFrame(irradiance, columns=wavelengths_str)
eve_lines.index = yyyydoy_sod_to_datetime(eve_readsav.yyyydoy, eve_readsav.sod)  # Convert EVE standard time to datetime

In [7]:
# Get GOES flare events above C1 within date range corresponding to EVE data
#flares = get_goes_flare_events(eve_lines.index[0], eve_lines.index[-1], verbose=verbose)  # TODO: The method in sunpy needs fixing, issue 2434

In [8]:
# prototype only: load GOES events from IDL saveset instead of directly through sunpy
goes_flare_events = readsav('/Users/jmason86/Dropbox/Research/Data/GOES/events/GoesEventsC1MinMegsAEra.sav')
goes_flare_events['class'] = goes_flare_events['class'].astype(str)
goes_flare_events['event_peak_time_human'] = goes_flare_events['event_peak_time_human'].astype(str)
goes_flare_events['event_start_time_human'] = goes_flare_events['event_start_time_human'].astype(str)
goes_flare_events['peak_time'] = Time(goes_flare_events['event_peak_time_jd'], format='jd', scale='utc')
goes_flare_events['start_time'] = Time(goes_flare_events['event_start_time_jd'], format='jd', scale='utc')
num_flares = len(goes_flare_events['class'])

In [9]:
# Start a progress bar
widgets = [progressbar.Percentage(), progressbar.Bar(), progressbar.Timer(), ' ', progressbar.AdaptiveETA()]
progress_bar = progressbar.ProgressBar(widgets=widgets, max_value=num_flares).start()
# just call progress_bar.update(i) to update this at later times

N/A%|                                     |Elapsed Time: 0:00:00 ETA:  --:--:--

In [10]:
# prototype only: import time just for example progress bar below
import time 

In [11]:
# Prepare a hold-over pre-flare irradiance value,
# which will normally have one element for each of the 39 emission lines
preflare_irradiance = None

# Start loop through all flares
for flare_index in range(num_flares):
    # all subsequent cells actually go here
    progress_bar.update(flare_index + 1)
    time.sleep(0.0002)

 97%|##################################### |Elapsed Time: 0:00:01 ETA:  0:00:00

In [12]:
# prototype only: reset flare_index to 1 (not 0) 
flare_index = 1

In [13]:
# Reset the flare interrupt flag
flare_interrupt = False

In [19]:
# Determine pre-flare irradiance
minutes_since_last_flare = (goes_flare_events['peak_time'][flare_index] - goes_flare_events['peak_time'][flare_index - 1]).sec / 60.0
if  minutes_since_last_flare > threshold_time_prior_flare_minutes:
    # Clip EVE data from threshold_time_prior_flare_minutes prior to flare up to peak flare time
    eve_lines_preflare_time = eve_lines[(goes_flare_events['peak_time'][flare_index] - (threshold_time_prior_flare_minutes * u.minute)).iso:
                                        (goes_flare_events['peak_time'][flare_index]).iso]
    
    # Loop through the emission lines and get pre-flare irradiance for each
    preflare_irradiance = []
    for column in eve_lines_preflare_time:
        eve_line_preflare_time = pd.DataFrame(eve_lines_preflare_time[column])
        eve_line_preflare_time.columns = ['irradiance']
        preflare_irradiance.append(determine_preflare_irradiance(eve_line_preflare_time,
                                                                 pd.Timestamp(goes_flare_events['start_time'][flare_index].iso),
                                                                 verbose=verbose))
else: 
    logger.info("This flare at {0} will use the pre-flare irradiance from flare at {1}."
                .format(goes_flare_events['peak_time'][flare_index].iso, 
                        goes_flare_events['peak_time'][flare_index - 1].iso))

2018-01-25 16:34:12,540 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-25 16:34:12,540 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-25 16:34:12,540 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-25 16:34:12,540 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-25 16:34:12,540 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-25 16:34:12,540 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-25 16:34:12,540 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-25 16:34:12,540 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16

2018-01-25 16:34:12,583 [INFO ] [determine_preflare_irradiance] Converted irradiance to percent, baselining median in entire pre-flare window.
2018-01-25 16:34:12,583 [INFO ] [determine_preflare_irradiance] Converted irradiance to percent, baselining median in entire pre-flare window.
2018-01-25 16:34:12,583 [INFO ] [determine_preflare_irradiance] Converted irradiance to percent, baselining median in entire pre-flare window.
2018-01-25 16:34:12,583 [INFO ] [determine_preflare_irradiance] Converted irradiance to percent, baselining median in entire pre-flare window.
2018-01-25 16:34:12,583 [INFO ] [determine_preflare_irradiance] Converted irradiance to percent, baselining median in entire pre-flare window.
2018-01-25 16:34:12,583 [INFO ] [determine_preflare_irradiance] Converted irradiance to percent, baselining median in entire pre-flare window.
2018-01-25 16:34:12,583 [INFO ] [determine_preflare_irradiance] Converted irradiance to percent, baselining median in entire pre-flare window.

2018-01-25 16:34:12,652 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-25 16:34:12,652 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-25 16:34:12,652 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-25 16:34:12,652 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-25 16:34:12,652 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-25 16:34:12,652 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-25 16:34:12,652 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-25 16:34:12,652 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-25 16:34:12,652 [INFO ] 

2018-01-25 16:34:12,685 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-25 16:34:12,685 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-25 16:34:12,685 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-25 16:34:12,685 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-25 16:34:12,685 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-25 16:34:12,685 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more th

2018-01-25 16:34:12,716 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-25 16:34:12,750 [INFO ] [determine_preflare_irradiance] Converted irradiance to percent, baselining median in entire pre-flare window.
2018-01-25 16:34:12,750 [INFO ] [determine_preflare_irradiance] Converted irradiance to percent, baselining median in entire pre-flare window.
2018-01-25 16:34:12,750 [INFO ] [determine_preflare_irradiance] Converted irradiance to percent, baselining median in entire pre-flare window.
2018-01-25 16:34:12,750 [INFO ] [determine_preflare_irradiance] Converted irradiance to percent, baselining median in entire pre-flare window.
2018-01-25 16:34:12,750 [INFO ] [determine_preflare_irradiance] Converted irradiance to percent, baselining median in entire pre-flare window.
2018-01-25 16:34:12,750 [INFO ] [determine_preflare_irradiance] Converted irradiance to percent, baselining median in entire pre-flare window.
2018-01-25 16:34

...

2018-01-25 16:34:22,945 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-25 16:34:22,945 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-25 16:34:23,014 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-25 16:34:23,014 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-25 16:34:23,014 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-25 16:34:23,014 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-25 16:34:23,014 [WARNI] [determine_preflare_irradiance

2018-01-25 16:34:23,014 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-25 16:34:23,014 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-25 16:34:23,014 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-25 16:34:23,014 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-25 16:34:23,014 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-25 16:34:23,014 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more th

2018-01-25 16:34:23,079 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-25 16:34:23,079 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-25 16:34:23,079 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-25 16:34:23,079 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-25 16:34:23,079 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-25 16:34:23,079 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-25 16:34:23,079 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-25 16:34:23,079 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16

2018-01-25 16:34:23,287 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-25 16:34:23,287 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-25 16:34:23,287 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-25 16:34:23,287 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-25 16:34:23,287 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-25 16:34:23,287 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-25 16:34:23,287 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-25 16:34:23,287 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-25 16:34:23,287 [INFO ] 

2018-01-25 16:34:23,349 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-25 16:34:23,349 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-25 16:34:23,349 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-25 16:34:23,349 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-25 16:34:23,349 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-25 16:34:23,349 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more th

In [25]:
preflare_irradiance

[None,
 None,
 None,
 5.8936072794798143e-05,
 3.774480865104124e-05,
 5.0253101411120347e-05,
 3.5352071184509747e-05,
 2.793670986041737e-05,
 1.5282487462779198e-05,
 3.0548522772733122e-05,
 None,
 0.00048761661552513641,
 None,
 None,
 4.0887623375359304e-05,
 None,
 None,
 1.137426837279539e-05,
 None,
 None,
 None,
 2.3469197913073003e-05,
 None,
 4.5233417040435597e-05,
 None,
 None,
 8.0574476063096281e-06,
 None,
 4.7459959508463122e-05,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]